In [1]:
import pickle

def dump(obj,filename):
    filehandler = open(filename,"wb")
    pickle.dump(obj,filehandler)
    filehandler.close()

def load(filename):
    file = open(filename,'rb')
    obj = pickle.load(file)
    file.close()
    return obj

In [2]:
def getCleanText(text, lower = True):
    text = str(text)
    parse_text = BeautifulSoup(text).get_text()
    
    letters_only = re.sub(r'http[s]?:\/\/(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' url ', 
                          parse_text, flags=re.MULTILINE)
    letters_only = re.sub('^(1?[0-9]|2[0-3]):[0-5][0-9]$','time',letters_only)
    letters_only = re.sub('(name|NAME)\d{1,}','username',letters_only)
    
    
    letters_only = re.sub("[^a-zA-Z0-9\.:']",  
                      " ",                   
                      letters_only)
    letters_only = letters_only.replace('\n'," ")
    letters_only = letters_only.replace('\r'," ")
    letters_only = re.sub('[.]{2,}', '. ', letters_only)
    
    
    if lower:
        letters_only = letters_only.lower()
  
    
    
    #words = word_tokenize(lower)
    words = CountVectorizer(stop_words='english').build_tokenizer()(letters_only)
    
    
    meaningful_words = [ w for w in words if len(w)> 0 and len(w)<20]
    
    
    #meaningful_words = [w if (w not in countries) and (w not in cities) else u"country" for w in meaningful_words]
    #lem_words = [wordnet_lemmatizer.lemmatize(w) for w in  meaningful_words]
    #stem_words = [snowball_stemmer.stem(w) for w in lem_words]
    
    
    
    clean_text = " ".join(meaningful_words)

    return clean_text

In [23]:
obj = load('orgs_clean.pkl')

In [24]:
print(list(obj))

[[u'<UNK>'], [[u'\U0001f354and a Diet Coke']], [[u'Zero Sugar', u'Coca-Cola', u'Scotch Whisky', u'Coke']], [[u'Coca-Cola Bottling Co.']], [u'<UNK>'], [[u'Taste\n Food', u'Diet', u'Sugar', u'Warmth\n Energy']], [[u'Pepsi-Cola', u'Playboy']], [[u'Britney, Beyonce']], [[u'Coke']], [[u'Pepsi']], [[u'AskarUpahan']], [[u'BritneySpears', u'Pepsi\u2019s', u'Beyonc\xe9', u'Super Bowl']], [u'<UNK>'], [[u'Pepsi']], [[u'PEPSI']], [[u'Coca Cola Zero']], [[u'Coca-Cola']], [[u'Pepsi', u'Mojo']], [u'<UNK>'], [[u'Blasted Crust Rolls', u'Pizza Sticks', u'Totino\u2019s Pizza Rolls', u'Coca-Cola', u'Keebler Chips Deluxe Cookies', u'Grooves Cracker Chips', u'9/29/17 (Publix Coupon Green Grocery Flyer', u'Rolls', u"Totino's", u'Stuffed Nachos', u'Coca-Cola Product', u'Cheez-It Baked Snack Crackers']], [[u'Pepsi']], [[u'Pepsi', u'Apple', u'UA', u'NFL', u'This Social Justice Warrior']], [[u'Pepsi', u'Amazed Velcro', u'Coke']], [[u'Drug', u'Supermarkets, Dollar', u'Coca Cola', u'Ikea, Liquor', u'Convenience St

In [37]:
for o in obj:
    print(o[0])

<UNK>
[u'\U0001f354and a Diet Coke']
[u'Zero Sugar', u'Coca-Cola', u'Scotch Whisky', u'Coke']
[u'Coca-Cola Bottling Co.']
<UNK>
[u'Taste\n Food', u'Diet', u'Sugar', u'Warmth\n Energy']
[u'Pepsi-Cola', u'Playboy']
[u'Britney, Beyonce']
[u'Coke']
[u'Pepsi']
[u'AskarUpahan']
[u'BritneySpears', u'Pepsi\u2019s', u'Beyonc\xe9', u'Super Bowl']
<UNK>
[u'Pepsi']
[u'PEPSI']
[u'Coca Cola Zero']
[u'Coca-Cola']
[u'Pepsi', u'Mojo']
<UNK>
[u'Blasted Crust Rolls', u'Pizza Sticks', u'Totino\u2019s Pizza Rolls', u'Coca-Cola', u'Keebler Chips Deluxe Cookies', u'Grooves Cracker Chips', u'9/29/17 (Publix Coupon Green Grocery Flyer', u'Rolls', u"Totino's", u'Stuffed Nachos', u'Coca-Cola Product', u'Cheez-It Baked Snack Crackers']
[u'Pepsi']
[u'Pepsi', u'Apple', u'UA', u'NFL', u'This Social Justice Warrior']
[u'Pepsi', u'Amazed Velcro', u'Coke']
[u'Drug', u'Supermarkets, Dollar', u'Coca Cola', u'Ikea, Liquor', u'Convenience Stores']
[u'Patriots']
[u'PepsiCo Inc.', u'Pepsi', u'Super Bowl', u'R&B', u'Betty Inc

[u'NFL', u'Pepsi']
[u'Coca-Coca', u'Coca-Cola', u'Atlanta Tribune', u'WNBA', u'General Manager']
[u'Cubanrum & #', u'Pepsi']
[u'Coke']
[u'Pepsi']
[u'Whole Foods', u'Lunch- Burger Lounge', u'Buffalo Wild']
[u'Pepsi']
[u'Coke Zero Sugar']
<UNK>
[u'Coca']
[u'Coca Cola']
[u'Pepsi']
<UNK>
[u'Apocalypse', u'Pepsi', u'McDonalds', u'Chrtistians']
<UNK>
<UNK>
<UNK>
[u'Pepsi', u'Black']
[u'Sun,', u'Coca-Cola', u'K-9 Mail']
[u'Pepsi']
<UNK>
<UNK>
[u'Coca Cola']
[u'the World Most Funny & Bizarre Facts', u'Coca-Cola']
[u'Sprite', u'Pepsi']
[u'Apocalypse', u'Pepsi', u'McDonalds', u'Chrtistians']
[u'Scotch', u'Coke']
[u'HIV', u'Coca-Cola', u'Lennox']
[u'Apocalypse', u'Pepsi', u'McDonalds', u'Chrtistians']
[u'BritneySpears', u'Pepsi\u2019s', u'Beyonc\xe9', u'Super Bowl']
[u'The Pepsi House', u'Pepsi Cola', u'Pepsi']
[u'Pepsi', u'Black']
[u'Hershey', u'Coca-Cola', u'Taco Bell']
[u'Pepsi Co.', u'the Burger King', u'School']
[u'Pepsi']
[u'Un RT']
[u'Apocalypse', u'Pepsi', u'McDonalds', u'Chrtistians']
[u

<UNK>
[u'Coca-Cola', u'Coke']
<UNK>
<UNK>
[u'NUM', u'this Department', u'Coca-Cola Company', u'AGE']
<UNK>
[u'AVOID DOUBLE CLAIMS', u'Coca-Cola Company']
<UNK>
[u'Britney, Beyonce']
<UNK>
[u'Pepsi']
[u'Times-Mail', u'Coca-Cola', u'the Lawrence County Museum of History']
[u'BtsArmy #Army']
[u'Pepsi']
[u'The Unbeaten Clash']
[u'Coca-Cola']
[u'The Ultimate Water Party Experience', u'Presold \U0001f3ab', u'Sound \U0001f50aDj Romeo \U0001f170dm', u'\U0001f52b Water Guns', u'MinuteMade #Schweppes']
[u'Coca-Cola Zero Sugar']
[u'Coke', u'Latina']
<UNK>
<UNK>
<UNK>
[u'Adobe', u'Coca-Cola', u'Apple', u'Interbrand\u2019s', u'Adidas', u'Starbucks']
<UNK>
<UNK>
[u'BritneySpears', u'Pepsi\u2019s', u'Beyonc\xe9', u'Super Bowl']
[u'Twix']
[u'BritneySpears', u'Pepsi\u2019s', u'Beyonc\xe9', u'Super Bowl']
<UNK>
[u'Coca Cola']
[u'Pepsi', u'TV Fuels Social Branded', u'Digital Native\u2019s Game', u"Digital Native's", u'TBS', u'Nintendo Switch']
[u'FIFA', u'Mean Joe Greene', u'Coca-Cola']
<UNK>
<UNK>
<UNK>

<UNK>
[u'Chevrolet', u'Dover International Speedway', u'Pepsi', u'Ford', u'Chevy', u'Toyota', u'NASCAR', u'Dover']
[u'Coca-Cola']
[u'Pepsi']
[u'#Pepsi']
[u'AOL', u'FrozenSand>']
[u'Coca-Cola', u'the Supreme Court', u'Transparency International Brazil', u'McDonald', u'Brazil\u2019s', u'JBS']
<UNK>
[u'Samsung Galaxy S8 Plus', u'Coca-Cola FREEZE']
[u'the Tesco Healthy Living']
[u'Twin Peaks S1:E6', u'House of Cards']
<UNK>
[u'BritneySpears', u'Pepsi\u2019s', u'Beyonc\xe9', u'Super Bowl']
[u'Small', u'Kentucky Fried Chicken', u'PepsiCo Wines & Spirits', u'Pepsi-Cola', u'Small\u2019s', u'Pizza Hut', u'General Motors', u'Frito-Lay', u'GM', u'PepsiCo']
[u'HCCB', u'Coca-Cola', u'CEO at']
<UNK>
[u'Pepsi', u'Coca-Cola', u'CNG', u'Hindi', u'PepsiCo']
[u'Coca-Cola Bottling Co.', u'Risk & Volatility Coca-Cola Bottling Co. Consolidated']
[u'Pepsi']
[u'Coca-Cola Bottling Co.', u'Coca-Cola', u'Company']
[u'Pepsi']
[u'the Company\u2019s', u'Coca-Cola', u'Monster Energy', u'The Company\u2019s']
[u'REALL

<UNK>
[u'Coke', u'Sugar']
[u'LIFEWTR', u'Pepsi', u'Lipton & Pure Leaf', u'PEPSI', u'Naked', u'Target', u'Target Deal Idea', u'Target Gift Card']
<UNK>
[u'Family Sports Center']
[u'Playboy']
[u'ABC 33/40 News', u'Pepsi', u'Brookwood Forest Elementary', u'Facebook\n Twitter', u'Weather Xtreme']
[u'D-Troop', u'Coca-Cola']
[u'Diet Cokes', u'Cook Political Report', u'House', u'White House']
[u'Coca-Cola']
[u'Pin Promising Review']
[u'Coke']
[u'Coca Cola Amatil']
[u'Coca Cola']
<UNK>
[u'BritneySpears', u'Pepsi\u2019s', u'Beyonc\xe9', u'Super Bowl']
[u'Coca-Cola']
[u'Coca']
[u'Pepsi']
[u'Pepsi', u'The Economic Times']
<UNK>
[u'SNAKE \U0001f355 FUNnel', u'|FUNnel Vision Vlog', u'Touch & Rhyme"', u'REAL FOOD CHALLENGE \U0001f34f SHARK', u'COLA BATH CHALLENGE', u'FUNnel Vision Music Video Vlog']
[u'Diet']
[u'Pepsi']
[u'Coca']
[u'IMO']
[u'AKA Doctors', u'Coca-Cola']
<UNK>
<UNK>
[u'Coca Cola']
[u'Diet Cokes']
[u'Auschwitz', u'the Chicago Cubs', u'Coca-Cola']
[u'Coca-Cola']
[u'Pepsi']
[u'the Zero S

...

[u'Bank Direct']
[u'Pepsi', u'Black']
[u'Coke']
[u'Pepsi', u'Black']
[u'Pepsi', u'Black']
<UNK>
[u'U.S Empire', u'Coca Cola']
[u'Pepsi', u"C'mon", u'REALLY', u'TruthHertz', u'Coke']
[u'BYU', u'Coke']
[u'Frito Lay', u'Ford', u'Anheuser-Busch', u'Proctor & Gamble', u'Microsoft', u'Pepsi Co.']
[u'Coca-Cola', u'Nestle', u'PEPSI', u'COKE', u'NESTLE', u'PepsiCo']
[u'PEPSI']
<UNK>
[u'\U0001f648schlimm', u'Bei', u'\xc4pfel', u'Eiklar', u'Pepsi Light -']
<UNK>
[u'Pepsi', u'Black']
<UNK>
<UNK>
[u'Coca Cola']
[u'Pepsi']
[u'Columbia', u'Tapatalk Click']
[u'WOW', u'IDEAL']
[u'AppleMouse', u'Coca-Cola']
<UNK>
[u'Pepsi', u'Black']
[u'IDR 55K.']
<UNK>
<UNK>
[u'Pepsi']
<UNK>
<UNK>
[u'Pepsi']
[u'Pepsi', u'Coca']
[u'New Corporate Brand Campaign Moves Beyond Soda', u"Coca-Cola's"]
<UNK>
[u'Pepsi', u'Black']
<UNK>
[u'PEPSI']
<UNK>
[u'Pepsi']
[u'Pepsi']
<UNK>
<UNK>
[u'Pepsi', u'Word']
[u'Coca-Cola']
[u'Coca-Cola']
<UNK>
<UNK>
[u'Coca-Cola']
<UNK>
[u'Pepsi', u'Black']
<UNK>
[u'Coca-Cola']
[u'BritneySpears', 

[u'Coca Cola Blak']
<UNK>
<UNK>
[u'AFTERGLOW']
<UNK>
[u'Coca-Cola', u'Ford', u'Nike', u'Apple', u'McDonald\u2019s', u'Microsoft', u'Ford Motor Company', u'Coke', u'RussianThis']
[u'Coca Cola', u'Chocolate', u'Coca']
[u'Coca-Cola']
[u'Pepsi Cola', u'NYC']
<UNK>
[u'Campbell\u2019s Soup Company', u'Dairy Management', u'Bose', u'Anheuser-Busch', u'\u2022 News America', u'Bridgestone', u'Inc. (', u'Extreme Networks', u'Visa', u'NFL', u'Procter & Gamble \u2022', u'Hyundai Motor America']
[u'VISA', u'Pepsi', u'Coca-Cola']
[u'PEPSI']
[u'Apocalypse', u'Pepsi', u'McDonalds', u'Chrtistians']
<UNK>
[u"UK Gov's", u'UK Gov\u2019s']
<UNK>
[u'Anheuser-Busch', u'Pepsi', u'Panama City Beach', u'Friends of Army']
[u'Coke']
<UNK>
[u'24 Lodging Partner', u'Pepsi Southern Originals', u'Florida Lottery Stage', u'The Pepsi Gulf Coast Jam', u'Pepsi Gulf Coast Jam', u'DSI Security Services']
[u'Pepsi', u'Coke']
<UNK>
<UNK>
<UNK>
<UNK>
[u'No Sugar Metamucil']
[u'AMELIE']
[u'Coke']
<UNK>
<UNK>
[u'Identity Theft N

[u'Pepsi Procter & Gamble Quaker Verizon', u'Anheuser-Busch Barclaycard', u'Inc. (', u'Marriott Dairy Management', u'Bose Bridgestone', u'NFL', u'Dannon Extreme Networks', u'Hyundai Motor America']
[u'Coca-Cola']
[u"Coca-Cola's", u'Coca-Cola', u'Coke']
[u'IMHO']
[u'Coca-Cola']
[u'Coca-Cola', u'USA TODAY NETWORK Fullscreen A', u'Rockport', u'Getty Images Fullscreen Search', u'Northjersey.com', u'USA TODAY NETWORK', u'IL', u'Getty Images Fullscreen Floodwater']
[u'Coca-Cola Sucks', u'Buy Bottled Water']
[u'Coca-Cola']
<UNK>
[u'Pepsi', u'Black']
[u'Mercedes-Benz', u'Coca-Cola', u'Apple', u'Adobe (', u'Adidas (', u'Toyota', u'Microsoft', u'Amazon', u'IBM', u'Facebook', u'Samsung']
[u'Coca-Cola', u'Coke Studio']
[u'Pepsi']
[u'PepsiCo']
[u'Coca-Cola']
[u'Pepsi', u'Black']
<UNK>
[u'AT&T', u'Pepsi', u"FOX'S", u'Ford', u'Kellogg', u'PledgeMusic', u'NBC', u"Bravo's", u'Heineken', u'Nissan', u'CBS', u"TLC's", u'Coke', u'Hyundai']
[u'Pepsi']
[u'Music', u'Coke']
[u'Pepsi']
[u'\u2019s Hellmann', u'C

[u'Midwesterners', u'Coca-Cola Classic', u'Coca-Cola', u'Coke']
[u'Pepsi', u'Beyonc\xe9']
[u'Pepsi', u'Beverage Digest', u'Coke']
[u'Coca Cola']
[u'generation\u201d\u2014as', u'Pepsi']
[u'Pepsi']
[u'Pepsi', u'Coke']
[u'CF Korean Model', u'Coca-Cola']
[u'Pepsi Challenge', u'Pepsi Paradox', u'Coke']
[u'Pepsi', u'VERIZON', u'Toyota', u'Bud Light', u'NFL']
[u'Pepsi', u'Baylor College Medicine', u'Coke']
[u'Denver Sports']
[u'New Coke', u'Pepsi', u'Coke']
[u'Coca Cola', u'Chocolate', u'Coca']
[u'Coke', u'The Pepsi Challenge']
[u'Coca']
[u'McDonald', u'Coke', u'Sugar']
[u'Marriott', u'Pepsi\u2019s Project Taste Test', u'NRAEF', u'Pepsi']
[u'Kraft', u'Committee', u'EU', u'Coca Cola', u'Kellogg', u'Sugar']
<UNK>
[u'Pepsi', u'the United Nations\u2019 Food and Agriculture Organization']
[u'P.S. Consumption', u'Coke', u"n't"]
[u'CCE', u'Institutional & Insider Ownership', u'Barfresh Food Group', u'Coca-Cola European Partners PLC', u'BRFH']
[u'Pepsi-Cola', u'Playboy']
[u'Coca-Cola European Partner

[u'Keds', u'the Rolling Stones', u'Apple']
[u'Coke']
<UNK>
[u'Pepsi']
[u'McCaf\xe9 Frappes', u"Facebook Conversations McDonald's", u'Continue Reading NEWSLETTER', u'McDonald', u'Coca']
<UNK>
[u'Pepsi', u'Bernie Boston']
[u'Auschwitz', u'the Chicago Cubs', u'Coca-Cola']
[u'Coke']
<UNK>
[u'Pepsi', u'Coke']
[u'PepsiCo', u'Coca-Cola']
[u'The Pennsylvania Merchants Association', u'PepsiCo', u'Coca-Cola']
<UNK>
<UNK>
[u'BritneySpears', u'Pepsi\u2019s', u'Beyonc\xe9', u'Super Bowl']
<UNK>
[u'Coca']
[u'Bottles & Aquafina Sparkling', u'Pepsi', u'Pumpkin Spice']
<UNK>
[u'WIP', u'Emerald']
[u'Apocalypse', u'Pepsi', u'McDonalds', u'Chrtistians']
<UNK>
<UNK>
[u"Coca-Cola's", u'PepsiCo']
<UNK>
[u'Coca']
[u'BritneySpears', u'Pepsi\u2019s', u'Beyonc\xe9', u'Super Bowl']
[u'Coca Cola', u'MISS']
[u'Retro Birthday Party\U0001f387']
<UNK>
<UNK>
[u'Pepsi']
[u'PTSD', u'THE WAY']
[u'Coca-Cola']
[u'Coca-Cola']
[u'Apocalypse', u'Pepsi', u'McDonalds', u'Chrtistians']
<UNK>
[u'Cootamundra -2590', u'Coke', u'Coca

In [68]:
pepsiIdx=[];
cokeIdx=[];
bothIdx=[];
i=0;
for o in obj:
    if(o[0] != "<UNK>"):
        foundCoke = False;
        foundPepsi = False;
        for ent in o[0]:
            print(ent)
            ent2=ent.lower()
            if "coke" in ent2 or "coca" in ent2 or "cola" in ent2:
                foundCoke = True;
            if "pepsi" in ent2 or "pepsico" in ent2:
                foundPepsi = True;
        print(": "+str(foundCoke)+" "+str(foundPepsi))
        print("\n")
        if foundCoke and foundPepsi:
            bothIdx.append(i)
        elif foundCoke:
            cokeIdx.append(i)
        elif foundPepsi:
            pepsiIdx.append(i)
    i=i+1;

🍔and a Diet Coke
: True False


Zero Sugar
Coca-Cola
Scotch Whisky
Coke
: True False


Coca-Cola Bottling Co.
: True False


Taste
 Food
Diet
Sugar
Warmth
 Energy
: False False


Pepsi-Cola
Playboy
: True True


Britney, Beyonce
: False False


Coke
: True False


Pepsi
: False True


AskarUpahan
: False False


BritneySpears
Pepsi’s
Beyoncé
Super Bowl
: False True


Pepsi
: False True


PEPSI
: False True


Coca Cola Zero
: True False


Coca-Cola
: True False


Pepsi
Mojo
: False True


Blasted Crust Rolls
Pizza Sticks
Totino’s Pizza Rolls
Coca-Cola
Keebler Chips Deluxe Cookies
Grooves Cracker Chips
9/29/17 (Publix Coupon Green Grocery Flyer
Rolls
Totino's
Stuffed Nachos
Coca-Cola Product
Cheez-It Baked Snack Crackers
: True False


Pepsi
: False True


Pepsi
Apple
UA
NFL
This Social Justice Warrior
: False True


Pepsi
Amazed Velcro
Coke
: True True


Drug
Supermarkets, Dollar
Coca Cola
Ikea, Liquor
Convenience Stores
: True False


Patriots
: False False


PepsiCo Inc.
Pepsi
Super B

Tri-State Metro
Coca-Cola
the Emergency Management Assistance Compact
: True False


DPR
Coca-Cola Company
: True False


World’s Highest-Paid TV
Head & Shoulders
Big Bang Theory
Forbes
: False False


Barclays PLC
Coca-Cola Company
Morningstar, Inc.
Morgan Stanley
: True False


Pepsi
: False True


Pepsi
: False True


First Command Bank
Coca-Cola Company
Daily Political
: True False


Coca-Cola Southwest Beverages
CCSWB
: True False


MarketBeat.com
Daily - Enter
Coca-Cola Company
Receive News & Ratings
Company
: True False


Auschwitz
the Chicago Cubs
Coca-Cola
: True False


Coca-Cola
Hattiesburg American Original
Coca-Cola Company's
The Bottling Company
Coca-Cola Plant
: True False


Coca-Cola
: True False


Coca-Cola Company's
The Bottling Company
Coca-Cola
Buy Photo The Bottling Company
: True False


Thomson
Coca-Cola Bottling Company
The Bottling Company
Coca-Cola
: True False


Experience’
Coca Cola TVCM
DNA
: True False


Pepsi
: False True


the U.S. Patent Office
Coca-Col

LTL Mayo
BBQ
Pepsi
: False True


Pepsi
: False True


Pepsi
: False True


Coca-Cola
: True False


Pepsi
: False True


Coca-Cola
: True False


Pepsi
Black
: False True


GMO
Pepsi
: False True


Chahinaz Seghiri February
Coca-Cola
Super Bowl
: True False


Pepsi
Black
: False True


Pepsi Procter & Gamble Quaker Verizon
Inc. Dannon
Marriott Dairy Management
Bose Bridgestone
NFL
Hyundai Motor America
: False True


Pepsi
: False True


Pepsi
Black
: False True


Michael and Luke
@MasBah5SOS & @robertazlima
: False False


Coke Zero Sugar
: True False


Pepsi
Dannon
Busch,
McDonalds
Gillette
Quaker Oats
: False True


Coca Cola Bottling Co. United Inc.
FSU
Clark
Seminole Dining
the Salvation Army
: True False


KFC
Pepsi
: False True


Coca-Cola
: True False


Coca
: True False


Coca-Cola
: True False


Pepsi
: False True


McDonald
Apple
Coca-Cola
Coke
: True False


Pepsi Fan Party
: False True


Coca
: True False


Pepsi
: False True


Sky News
HIV
Pepsi
: False True


Pepsi
BYU


: True False


NFL
Pepsi
: False True


Coca-Cola
: True False


Coca
: True False


CHR
: False False


BP ⬆️
: False False


NFL
Pepsi
NEVER
: False True


BritneySpears
Pepsi’s
Beyoncé
Super Bowl
: False True


Coca Cola
Amazon
Pepsi
: True True


NFL
Pepsi
Coke
: True True


Best Before
Pepsi
: False True


Next Generation Marketing
: False False


Coca Cola
Coca-Cola
: True False


Twin Tower
Chanwoo x Junhoe
Pepsi Fan Party
: False True


Coca Cola
: True False


Pepsi
: False True


Diet
’s
BOY
: False False


BritneySpears
Pepsi’s
Beyoncé
Super Bowl
: False True


PEPSI
: False True


Wolf Tracker
: False False


Pepsi
: False True


Coca-Cola
: True False


Pepsi
: False True


Red Bull
Coca Cola
: True False


the People’s Choice Award
Toro
Q&A
TIFF
Best Actress
: False False


Pepsi
: False True


Charing
: False False


Pizza
Burrito
Coca Cola
Ice Cream
: True False


Alzheimer's
Coke
: True False


Coca-Cola
: True False


Android
Pepsi
’s
Apple
Coke
: True True


Coke
Coc

ABC
: False True


Email job Reference
Coca- Cola European Partners
33359231 Bank
: True False


Pepsi
Coca-Cola
United
Beverage Digest'
PepsiCo
Coca
Carbonated Soft Drinks
: True True


Coke
: True False


Coke
: True False


General Electric
Eastman Kodak
Coca-Cola
Public Affairs
: True False


the Associated Press
Cahillane
Frosted Flakes
the Clinton Global Initiative America
Coca-Cola Americas
Pop-Tarts
Strawberry
FILE
Kellogg
: True False


BritneySpears
Pepsi’s
Beyoncé
Super Bowl
: False True


Pepsi
Black
: False True


Pepsi
: False True


Coke
: True False


PepsiCo
Electronic Art's
Pepsi
Kendall Jenner
Coke
: True True


Coca-Cola
: True False


Coke
: True False


Pepsi Cola Bottling Company
Pepsi Cola Football
Atmore News
: True True


Dr Pepper
Sun Chips
Twizzlers
Cheetos
: False False


Disney
’s Creative
NBA
Coca Cola
Nike
Champion
Bergdorf Goodman
Moncler
: True False


USA TODAY Sports Ryan Blaney
Pocono
Dover International Speedway
Coca-Cola
USA TODAY Sports Eventual


...



BritneySpears
Pepsi’s
Beyoncé
Super Bowl
: False True


Travel +
: False False


@Ford
Blue
NFL
BIGLY
Red,
: False False


Coca-Cola
: True False


Pepsi
OrderQuantity
OrderName
Customers C
CutomerID | OrderName
CSV
: False True


No Caffeine
Coke
: True False


Tryon International Equestrian Center
Fall Dressage
Adequan®
Coca-Cola®
the Legends Club
CDI-W
TIEC
: True False


WIN
Superdrug
IOM
Boots
: False False


Pepsi
Black
: False True


Coca-Cola
Saints
Coca-Cola Bottling Company United
Bayou
Coke
Super Bowls
: True False


Delta Air Lines
The Home Depot
Coca-Cola Company
Economy Main
Rubbermaid
Newell
AT&T Mobility
UPS
: True False


Coca-Cola
: True False


Pepsi
Coke
Commercial Banks
: True True


Coke
: True False


Coca Cola
Chocolate
Coca
: True False


FEMA
Coca-Cola
: True False


Coca-Cola
Monster Energy Drink
Almirola
SHR
Smithfield
: True False


ICC
the New York Times
: False False


Azure Mobile Services
Customers
: False False


San Pelegrino
: False False


EVIO
CO


Pepsi
: False True


KENNEKAMARTIN |
@craig.clarkproductions
: False False


Pepsi
: False True


Coca-Cola Australia
: True False


OPK
DH
: False False


Pepsi-Russia
Pepsi
: False True


Sugar Cookies
Coke
: True False


Anti-Soda Monopoly
Pepsi
: False True


Coca-Cola
: True False


Pepsi Special Drink
Pepsi Special Pepsi
: False True


REALLY
: False False


Coca Cola
: True False


Spilt Pepsi
Myself - HELP
NEVER
: False True


Front
Benetton
Coca-Cola
: True False


Coca-Cola
: True False


Coca-Cola
Gilbert Barbarian
Black
: True False


Pepsi
: False True


Coke
: True False


BritneySpears
Pepsi’s
Beyoncé
Super Bowl
: False True


Pepsi
: False True


BYU
Coke
: True False


the American Red Cross
CNN
Coca-Cola
: True False


The Athletic (Toronto)
Argonauts
Pepsi
Simplicity Financial Marketing Acquires
Simplicity Financial Marketing
DoubleTree
AdNews Here's
the CBC Radio
: False True


Power Rangers
Pepsi
Stitch
: False True


Pepsi
: False True


Coke
: True False


Pepsi

Chrysler
Coca-Cola Company
SapientNitro’s
Target
Lufthansa,
Citi
Vodafone
: True False


Mug Cream Soda
Coca Cola
•∞ Freedom Nuka Wild
: True False


Coca-Cola
New Coke’.
: True False


Coca Cola
: True False


All Inc
The All Inc
: False False


Water = Health
: False False


Coca-Cola Bottlers Business Services
: True False


Coca-Cola
Unilever,
Nestlé
General Mills
Danone
Kellogg
PepsiCo
: True True


Pepsi
: False True


BritneySpears
Pepsi’s
Beyoncé
Super Bowl
: False True


AR & #
: False False


Coca-Cola
Nike
The Works
Google’s
Samsung
: True False


Diet Cokes
: True False


Jollibee Philippines Marketing
Coca-Cola Philippines Vice
Jollibee Philippines Human Resources
Coca-Cola
Jollibee Group Foundation
JFVA
VillarinActress Dimples
OFW
Public Affairs and Communications Atty
Php
Jollibee Global Brand
ANC
: True False


GMT-Master
Pepsi
: False True


Coca-Cola
: True False


SMH
: False False


Coca Cola
Mean Joe Greene
: True False


BritneySpears
Pepsi’s
Beyoncé
Super Bowl
: 



Coca Cola
: True False


Coca
: True False


Apple
Coca Cola
Wells Fargo
American Express
Bank of America
: True False


Pepsi
Coke
: True True


Lego
: False False


Coca-Cola
: True False


Cuisine of
the Georgia Aquarium,[160
the Brooklyn-
The World of Coca-Cola
Tourism Main
Indie
OutKast
: True False


Coca
: True False


NFL
Pepsi
: False True


BritneySpears
Pepsi’s
Beyoncé
Super Bowl
: False True


the Partido Acción Nacional
PAN
PRI
Coca-Cola
: True False


BritneySpears
Pepsi’s
Beyoncé
Super Bowl
: False True


BritneySpears
Pepsi’s
Beyoncé
Super Bowl
: False True


SP
: False False


Santa Claus’s
Coca-Cola
: True False


Pepsi
Beyoncé
: False True


Coca
: True False


Muse
Sun
Pepsi
: False True


Coca-Cola
AB InBev
Coke
: True False


AtifAslam
: False False


Hartsfield-Jackson
CSX
Coca-Cola
Norfolk Southern
: True False


Omnicom Group
Coca-Cola's
Coca-Cola
: True False


Mercedes-Benz
Coca-Cola
Apple
Adobe (
Toyota
Microsoft
IBM
Adidas
Samsung
: True False


Diet
Diet


PTSD
THE WAY
: False False


Coca-Cola
: True False


Coca-Cola
: True False


Apocalypse
Pepsi
McDonalds
Chrtistians
: False True


Cootamundra -2590
Coke
Coca Cola Coke
: True False


COCA-COLA Beverages South Africa
CCBSA
Coca-Cola Beverages
the Ocean Conservancy
the International Coastal Clean-
Centre
: True False


Apocalypse
Pepsi
McDonalds
Chrtistians
: False True


Langley
Polly Brewer
Coca Cola
: True False


Coca-cola Taste
Coca
: True False


BritneySpears
Pepsi’s
Beyoncé
Super Bowl
: False True


FIFA
Havas SE's
Havas
Hyundai-Kia
Havas Media
Havas SE
Havas Worldwide
Jérôme
: False False


Coca-Cola
: True False


Coca-Cola
: True False


Coca-Cola
Taco Bell
: True False


PepsiFire
: False True


Pepsi
: False True


Remedios Peal Marama
the Revise Penal Code
Coca-Cola
: True False


DGI
Coca-Cola
NYSE
: True False


Coca-cola
: True False


Fanta World
: False False


Sprite,
Coke
: True False


Volkswagen
Google’s YouTube
Coca-Cola
Johnson & Johnson
: True False


Coca C

In [69]:
print(pepsiIdx)
print(cokeIdx)
print(bothIdx)

[9, 11, 13, 14, 17, 20, 21, 26, 28, 39, 45, 52, 53, 55, 62, 79, 82, 95, 100, 122, 130, 135, 141, 147, 161, 162, 166, 171, 176, 179, 183, 189, 190, 191, 192, 196, 201, 211, 214, 216, 221, 223, 224, 227, 248, 250, 257, 267, 268, 269, 282, 283, 290, 293, 299, 305, 308, 309, 313, 322, 329, 331, 339, 340, 342, 355, 356, 358, 365, 373, 376, 378, 380, 382, 387, 389, 393, 395, 400, 401, 404, 405, 407, 409, 410, 412, 426, 435, 436, 450, 454, 456, 457, 458, 460, 465, 470, 471, 474, 481, 489, 490, 494, 495, 510, 513, 514, 515, 517, 519, 531, 535, 537, 548, 549, 550, 557, 558, 565, 576, 579, 580, 581, 582, 585, 586, 593, 594, 601, 603, 605, 616, 620, 626, 633, 636, 640, 643, 652, 657, 662, 665, 669, 673, 675, 684, 685, 686, 692, 696, 697, 702, 704, 706, 707, 708, 710, 715, 716, 719, 720, 723, 725, 730, 731, 733, 741, 742, 744, 753, 754, 756, 759, 767, 768, 769, 771, 774, 775, 777, 778, 779, 781, 785, 787, 793, 795, 799, 800, 805, 809, 810, 819, 827, 829, 832, 834, 836, 840, 841, 849, 851, 854, 858

In [61]:
string = 'pepsi coca-cola'
foundCoke=False;
foundPepsi=False;
if "coke" in string or "coca" in string or "cola" in string:
    foundCoke = True;
if "pepsi" in string or "pepsico" in string:
    foundPepsi = True;
if foundCoke and foundPepsi:
    print('yes')

yes


False
